# Bayesian MBG estimation

## Load preselected variables

## Geostatistical Modeling

In [1]:
import pymc as pm
#import aesara.tensor as at
#from aesara.graph.basic import Constant
import os
import pickle
import numpy as np
import arviz as az
import pandas as pd


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import StandardScaler
import pytensor.tensor as at

import uuid

In [2]:
# Get the number of CPU cores to max out the machine in the traning stage
num_cores = os.cpu_count()

print(f"Number of CPU cores: {num_cores}")

Number of CPU cores: 8


### Parameters

In [3]:
target_indicator = 'mpi'

In [4]:
#Define the directory where the pickle files are stored
pickle_dir = 'temp_files'

### Load the target and the covariates

In [5]:
# Load coordinates from the pickle file
with open(os.path.join(pickle_dir, 'coordinates.pkl'), 'rb') as f:
    coordinates = pickle.load(f)

# Load coordinates for observed rows from the pickle file
with open(os.path.join(pickle_dir, 'coordinates_observed.pkl'), 'rb') as f:
    coordinates_observed = pickle.load(f)

In [6]:
df = pd.read_pickle('temp_files/selected_features.pkl')

In [7]:
selected_features = df.columns.to_list()

# Remove target_values and others
remove_list = [target_indicator, 'geometry', 'grid_id']

# Remove elements in remove_list from main_list
selected_features = [item for item in selected_features if item not in remove_list]

In [8]:
#Only rows with observed target indicator
df1 = df[~df[target_indicator].isnull()]

In [9]:
df1.shape

(105, 10)

In [10]:
# Covariate matrix
X = df1[selected_features].values

# Series with the target variable observed
response = df1[target_indicator].values

In [11]:
# Standardize features and response
X = (X - X.mean(axis=0)) / X.std(axis=0)
response = (response - response.mean()) / response.std()

### Predictions

Key Components of the Model

    Priors:
        beta: Coefficients for the linear model, assumed to follow a normal distribution with mean 0 and standard deviation 1.
        sigma: Standard deviation of the observation noise, assumed to follow a half-normal distribution with standard deviation 1.
        ls: Length-scale parameter for the spatial covariance function, assumed to follow a half-Cauchy distribution with scale parameter 1.

    Spatial Distance Matrix:
        D: Matrix of Euclidean distances between all pairs of observed locations.

    Covariance Function:
        K: Covariance function (Matern 5/2) which defines the spatial correlation structure.

    Gaussian Process (GP):
        gp: Latent Gaussian process with the defined covariance function.
        f: Prior distribution of the GP evaluated at the observed coordinates.

    Linear Model:
        mu: Mean of the linear model which is a combination of the linear predictor (X * beta) and the spatial effect (f).
        y_obs: Observed responses, modeled as a normal distribution with mean mu and standard deviation sigma.

    Inference:
        Using Automatic Differentiation Variational Inference (ADVI) to approximate the posterior distribution of the model parameters.
        advi_fit: Fitting the model using ADVI.
        trace: Sampling from the fitted model to obtain posterior samples.

In [12]:
%%time
# Fit a Bayesian geostatistical model
with pm.Model() as model:
    # Priors
    beta = pm.Normal('beta', mu=0, sigma=1, shape=len(selected_features))
    sigma = pm.HalfNormal('sigma', sigma=1)
    ls = pm.HalfCauchy('ls', beta=1)

    print('Priors run')

    # Spatial distance matrix
    D = np.sqrt(((coordinates_observed[:, None, :] - coordinates_observed[None, :, :])**2).sum(axis=-1))

    print('Distance matrix calculated')

    # Define covariance function
    K = pm.gp.cov.Matern52(2, ls=ls)
    gp = pm.gp.Latent(cov_func=K)
    f = gp.prior('f', X=coordinates_observed)

    print('Covariance run')

    # Linear model
    ## This defines the mean of the normal distribution for the observed data. It combines a linear regression term (pm.math.dot(X, beta)) with the GP latent function f.
    mu = pm.math.dot(X, beta) + f

    ## This defines the likelihood of the observed data (response) as a normal distribution with mean mu and standard deviation sigma.
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=response)

    print('Linear model specified')

    # Inference
    step = pm.NUTS(target_accept=0.95)
    idata = pm.sample(1000, tune=1000, step=step, cores=num_cores, return_inferencedata=True) #The num_cores parameter maxes the machine out. Tweak if needed. 

    print('Model Fitted')


Priors run
Distance matrix calculated
Covariance run
Linear model specified


Multiprocess sampling (8 chains in 8 jobs)
NUTS: [beta, sigma, ls, f_rotated_]


Output()

Sampling 8 chains for 1_000 tune and 1_000 draw iterations (8_000 + 8_000 draws total) took 331 seconds.
There were 266 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Model Fitted
CPU times: user 22.8 s, sys: 1.95 s, total: 24.7 s
Wall time: 5min 37s


In [13]:
#This bit is to save the trained model in case is needed
# Save the InferenceData (trace)
#trace_filename = 'model_trace.nc'

#az.to_netcdf(idata, trace_filename)
# Load the trace
#trace_filename = 'model_trace.nc'
#loaded_trace = az.from_netcdf(trace_filename)

#print("Trace successfully loaded")

## Generating predictions for new grids

In [14]:
# Filter out rows with NaN values in target indicator
df2 = df[df[target_indicator].isnull()] # Only unobserved rows

# Covariate matrix
X_new = df2[selected_features].values

In [15]:
# Standardize the new data using the same scaler fitted on the observed data
X_new = (X_new - X.mean(axis=0)) / X.std(axis=0)

# Load coordinates for the new locations
with open(os.path.join(pickle_dir, 'coordinates_unobserved.pkl'), 'rb') as f:
    coordinates_new = pickle.load(f)

# Ensure the scaler is fitted on the same data
scaler = StandardScaler().fit(df1[selected_features].values)
X_new = scaler.transform(df2[selected_features].values)

### Check if the covariance matrix is PSD
- PSD: Positive Semi-Definitive

In [24]:
import numpy as np
import pymc as pm
import pytensor.tensor as at

#This function is key to diagnose what is going on inside the Gaussian process

def diagnose_covariance_matrix(cov, jitter=1e-6):
    """
    Diagnose potential issues with a covariance matrix and suggest possible remedies.

    Parameters:
    ----------
    cov : np.ndarray or pytensor.tensor
        The covariance matrix to diagnose.
    jitter : float, optional
        The amount of jitter to add to the diagonal of the covariance matrix for stabilization.

    Returns:
    -------
    None
    """

    # Convert pytensor tensor to numpy array for diagnosis if necessary
    if isinstance(cov, at.TensorVariable):
        # Use pm.draw to evaluate the tensor as a NumPy array
        cov = pm.draw(cov)

    # Check for symmetry
    if not np.allclose(cov, cov.T):
        print("Warning: Covariance matrix is not symmetric.")
    else:
        print("Covariance matrix is symmetric.")

    # Check for positive semi-definiteness using eigenvalues
    eigvals = np.linalg.eigvalsh(cov)
    if np.all(eigvals >= 0):
        print("Covariance matrix is positive semi-definite (PSD).")
    elif np.all(eigvals > 0):
        print("Covariance matrix is positive definite (PD).")
    else:
        print("Covariance matrix is not positive semi-definite (non-PSD).")
        print("Eigenvalues:")
        print(eigvals)

    # Check for small or negative eigenvalues
    if np.any(eigvals < 0):
        print("There are negative eigenvalues, indicating non-PSD matrix.")
    elif np.any(eigvals == 0):
        print("There are zero eigenvalues, indicating the matrix is singular or nearly singular.")
    if np.any(eigvals < jitter):
        print("Some eigenvalues are smaller than the jitter value. Consider increasing jitter.")

    # Check the condition number (ratio of max to min eigenvalue)
    cond_number = np.linalg.cond(cov)
    print(f"Condition number of the matrix: {cond_number:.2e}")
    if cond_number > 1e10:
        print("Warning: Covariance matrix is ill-conditioned (large condition number).")
        print("Consider regularization or using a different covariance function.")

    # Suggest adding jitter and re-check PSD
    cov_with_jitter = cov + jitter * np.eye(cov.shape[0])
    eigvals_with_jitter = np.linalg.eigvalsh(cov_with_jitter)
    if np.all(eigvals_with_jitter >= 0):
        print("Adding jitter made the covariance matrix positive semi-definite.")
    else:
        print("Even after adding jitter, the matrix is still not positive semi-definite.")

    # Check for numerical issues using Cholesky decomposition
    try:
        np.linalg.cholesky(cov)
        print("Cholesky decomposition succeeded: Covariance matrix is positive definite.")
    except np.linalg.LinAlgError:
        print("Cholesky decomposition failed: Covariance matrix is not positive definite.")

    print("\nDiagnosis Complete.")

def generate_predictions(model, coordinates_new, X_new, idata, initial_jitter=1e-6, max_attempts=5):
    """
    Generate predictions for new data using a Gaussian Process model.

    Parameters:
    ----------
    model : pm.Model
        The PyMC model object that contains the Gaussian Process.
    coordinates_new : np.ndarray
        An array of coordinates for the new data points where predictions are needed.
    X_new : np.ndarray
        The covariate matrix for the new data points.
    idata : az.InferenceData
        The InferenceData object containing posterior samples from the fitted model.
    initial_jitter : float, optional
        The initial jitter value to add to the covariance matrix to ensure positive definiteness.
    max_attempts : int, optional
        Maximum number of attempts to find a stable jitter value.

    Returns:
    -------
    np.ndarray
        An array of mean predictions for the new data points.
    """

    with model:
        for attempt in range(max_attempts):
            try:
                jitter = initial_jitter * (10 ** attempt)
                unique_name = "f_pred_" + str(uuid.uuid4())

                # Generate the conditional GP with added jitter to the covariance matrix
                f_pred = gp.conditional(unique_name, coordinates_new, jitter=jitter)

                # Compute the mean of the beta samples
                beta_mean = idata.posterior['beta'].mean(dim=("chain", "draw")).values

                # Predictive mean
                mu_pred = pm.math.dot(X_new, beta_mean) + f_pred

                # Create the covariance matrix using PyMC's Matern32
                cov = pm.gp.cov.Matern32(coordinates_new.shape[1], ls=1.0)(coordinates_new)

                # Add jitter using Pytensor's identity matrix
                cov += jitter * at.eye(cov.shape[0])

                # Symmetrize the covariance matrix to ensure symmetry
                cov = (cov + cov.T) / 2

                # Check cov_matrix before predictions
                diagnose_covariance_matrix(cov)
                
                # Check for positive definiteness using Cholesky decomposition
                _ = at.slinalg.cholesky(cov)

                # If successful, proceed with prediction
                pred_samples = pm.sample_posterior_predictive(idata, var_names=[unique_name], return_inferencedata=True)
                return pred_samples.posterior_predictive[unique_name].mean(axis=0)

            except Exception as e:
                if attempt == max_attempts - 1:
                    raise ValueError(f"The covariance matrix is not positive semi-definite even after {max_attempts} attempts with increasing jitter.") from e


In [ ]:
# Generate predictions
mu_pred = generate_predictions(model, coordinates_new, X_new, idata)

# Transform the predictions back to the original scale
predicted_response = (mu_pred * response.std()) + response.mean()

Covariance matrix is symmetric.
Covariance matrix is positive semi-definite (PSD).
Condition number of the matrix: 1.16e+07


Sampling: [f_pred_6b367b46-6a07-4780-9296-e2bed9c93d87]


Output()

Covariance matrix is symmetric.
Covariance matrix is positive semi-definite (PSD).
Condition number of the matrix: 9.45e+06
Adding jitter made the covariance matrix positive semi-definite.
Cholesky decomposition succeeded: Covariance matrix is positive definite.

Diagnosis Complete.


Sampling: [f_pred_f049a6a4-d3b2-4ec7-89ca-2041568fe2ad]


Output()

Covariance matrix is symmetric.
Covariance matrix is positive semi-definite (PSD).
Condition number of the matrix: 3.32e+06
Adding jitter made the covariance matrix positive semi-definite.
Cholesky decomposition succeeded: Covariance matrix is positive definite.

Diagnosis Complete.


Sampling: [f_pred_b352282d-1c5a-4c93-ac14-6b0c96c69866]


Output()

In [ ]:
predicted_response = pd.DataFrame(predicted_response.transpose())

In [ ]:
# Create a DataFrame to store the results
df3 = pd.concat([df2['grid_id'].reset_index(drop=True), predicted_response], axis=1)

## Testing the model results

1. Posterior Predictive Checks
2. Prediction Accuracy Metrics
3. Residual Analysis
4. Uncertainty Quantification

### Posterior Predictive Checks:

Posterior Predictive Distribution: Compare the observed data to the posterior predictive distribution of the model. This involves generating new data based on the posterior distributions of the model parameters and comparing these simulated data to the actual observations.


In [ ]:
# Generate posterior predictive samples for checks
with model:
    posterior_predictive = pm.sample_posterior_predictive(idata, var_names=['y_obs'], return_inferencedata=True)

# Plot posterior predictive checks
az.plot_ppc(posterior_predictive, kind='kde', data_pairs={'y_obs': 'y_obs'})

# Show the plot
plt.show()

In [ ]:
az.plot_forest(idata, var_names=["beta"], combined=True, hdi_prob=0.95, r_hat=True)

### Prediction Accuracy Metrics

#### Mean Absolute Error (MAE)

Measures the average magnitude of the errors in a set of predictions, without considering their direction.

#### Root Mean Squared Error (RMSE) 

Measures the square root of the average of squared differences between predicted and observed values, providing an indication of the model’s overall error.

#### Coverage Probability

The proportion of observed data points that lie within a specified credible interval (e.g., 95% credible interval) of the predicted distribution. High coverage indicates that the model’s uncertainty estimates are reliable.

### Residual Analysis

#### Spatial Residual Plots 

Plot residuals (the differences between observed and predicted values) over the spatial domain to check for patterns. Randomly distributed residuals indicate a good fit.

In [ ]:
# Extract the observed and simulated data
y_obs = response
y_sim = posterior_predictive['y_obs'].mean(axis=0)  # Mean prediction for each observed data point

# Calculate residuals
residuals = y_obs - y_sim

# Plot spatial residuals
plt.figure(figsize=(12, 8))
sc = plt.scatter(coordinates_observed[:, 0], coordinates_observed[:, 1], c=residuals, cmap='coolwarm', s=100)
plt.colorbar(sc, label='Residuals')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Spatial Residual Plot')
plt.show()

# Histogram of residuals
plt.figure(figsize=(12, 6))
plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.show()

### Uncertainty Quantification

	•	Credible Intervals: Evaluate the width of the credible intervals for predictions. Narrower intervals indicate higher precision, but they should still encompass the true values.
	•	Uncertainty Maps: Generate maps of prediction uncertainty to visualize areas of high and low certainty in the predictions.

#### Credible Intervals

Evaluate the width of the credible intervals for predictions. Narrower intervals indicate higher precision, but they should still encompass the true values.

In [ ]:
# Extract the observed and simulated data
y_obs = response

# Generate posterior predictive samples for checks
with model:
    posterior_predictive = pm.sample_posterior_predictive(idata, var_names=['y_obs'], return_inferencedata=True)

# Extract the mean prediction, lower and upper bounds of the 95% credible intervals
y_sim = posterior_predictive.posterior_predictive['y_obs'].mean(dim=("chain", "draw")).values
hdi = az.hdi(posterior_predictive.posterior_predictive, hdi_prob=0.95)['y_obs']

# Calculate the width of the credible intervals
ci_width = hdi[:, 1] - hdi[:, 0]

# Check how many true values are within the credible intervals
within_ci = np.sum((y_obs >= hdi[:, 0]) & (y_obs <= hdi[:, 1]))
total_obs = len(y_obs)
coverage = within_ci / total_obs

print(f"Coverage of 95% Credible Intervals: {coverage * 100:.2f}%")

# Plot the credible intervals vs the observed values
plt.figure(figsize=(12, 8))
plt.errorbar(np.arange(len(y_obs)), y_sim, yerr=[y_sim - hdi[:, 0], hdi[:, 1] - y_sim], fmt='o', label='Predictions with 95% CI')
plt.plot(np.arange(len(y_obs)), y_obs, 'r.', label='Observed Values')
plt.xlabel('Data Point Index')
plt.ylabel('Values')
plt.title('Predictions with 95% Credible Intervals vs Observed Values')
plt.legend()
plt.show()

# Plot the width of the credible intervals
plt.figure(figsize=(12, 6))
plt.hist(ci_width, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Width of 95% Credible Intervals')
plt.ylabel('Frequency')
plt.title('Histogram of the Width of 95% Credible Intervals')
plt.show()

#### Credible intervals for predictions

In [ ]:
# Function to generate predictions using posterior samples
def generate_predictions(model, coordinates_new, X_new, idata):
    with model:
        jitter = 1e-5  # Increase the jitter value
        f_pred = gp.conditional('f_pred', coordinates_new, jitter=jitter)
        beta_mean = idata.posterior['beta'].mean(dim=("chain", "draw")).values
        mu_pred = pm.math.dot(X_new, beta_mean) + f_pred
        pred_samples = pm.sample_posterior_predictive(idata, var_names=["f_pred"], return_inferencedata=True)
        return mu_pred, pred_samples

# Generate predictions
mu_pred, pred_samples = generate_predictions(model, coordinates_new, X_new, idata)

In [ ]:
# Extract the samples for the predicted function values
f_pred_samples = pred_samples.posterior_predictive["f_pred"]

# Compute the 95% credible intervals
hdi = az.hdi(f_pred_samples, hdi_prob=0.95)

# Extract mean predictions
mu_pred_mean = f_pred_samples.mean(dim=("chain", "draw")).values

# Extract lower and upper bounds of the credible intervals
ci_lower = hdi.sel(hdi="lower").to_array().values.flatten()
ci_upper = hdi.sel(hdi="higher").to_array().values.flatten()

# Calculate the width of the credible intervals
ci_width = ci_upper - ci_lower

# Plot the credible intervals for new data
plt.figure(figsize=(12, 8))
plt.errorbar(np.arange(len(mu_pred_mean)), mu_pred_mean, yerr=[mu_pred_mean - ci_lower, ci_upper - mu_pred_mean], fmt='o', label='Predictions with 95% CI')
plt.xlabel('Data Point Index')
plt.ylabel('Predicted Values')
plt.title('Predictions with 95% Credible Intervals for New Data')
plt.legend()
plt.show()

# Plot the width of the credible intervals
plt.figure(figsize=(12, 6))
plt.hist(ci_width, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Width of 95% Credible Intervals')
plt.ylabel('Frequency')
plt.title('Histogram of the Width of 95% Credible Intervals for New Data')
plt.show()

# Generate uncertainty maps
plt.figure(figsize=(12, 8))
sc = plt.scatter(coordinates_new[:, 0], coordinates_new[:, 1], c=ci_width, cmap='coolwarm', s=100)
plt.colorbar(sc, label='Credible Interval Width')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Prediction Uncertainty Map')
plt.show()

In [ ]:
ci_upper

#### Uncertainty Maps 

Generate maps of prediction uncertainty to visualize areas of high and low certainty in the predictions.

#### Trace plots

Interpretation of Trace Plots

    Density Plots (Left Column):
        Each subplot on the left shows the kernel density estimate of the posterior distribution for a parameter.
        These plots give an idea of the central tendency (mean or median) and the spread (variance) of the parameter estimates.
        For example, the density plot for beta shows multiple colored curves corresponding to different chains, indicating the posterior distributions of the coefficients.

    Trace Plots (Right Column):
        Each subplot on the right shows the sampled values of the parameter across iterations for each chain.
        These plots help in assessing the convergence of the Markov Chain Monte Carlo (MCMC) sampling.
        A good trace plot should look like a "hairy caterpillar," with the chains mixing well and no apparent trends or patterns over iterations.

In [ ]:
import matplotlib.pyplot as plt

# Generate the trace plot
trace_plot = az.plot_trace(idata)

# Save the plot to a file
plt.savefig('temp_files/trace_plot.png')

### Create modeled surface maps

In [ ]:
# Assuming new_coordinates is a grid, reshape mean predictions
x_new = coordinates_new[:, 0].reshape(grid_shape)  # Define grid_shape as per your coordinate grid
y_new = coordinates_new[:, 1].reshape(grid_shape)
z_pred_mean = y_pred_mean.reshape(grid_shape)

# Plot the mean predictions
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x_new, y_new, z_pred_mean, cmap='viridis')

plt.title('Modeled Surface Map')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
ax.set_zlabel('Predicted Value')
plt.show()

### Create uncertainty maps

In [ ]:
# Reshape standard deviation predictions
z_pred_std = y_pred_std.reshape(grid_shape)

# Plot the uncertainty (standard deviation)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x_new, y_new, z_pred_std, cmap='viridis')

plt.title('Uncertainty Map')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
ax.set_zlabel('Standard Deviation')
plt.show()